In [163]:
import csv
import numpy as np
import scipy
from nltk.tokenize import sent_tokenize
import time

In [164]:
def makeDictionary(wordTokens, cutoff=0):
    '''
    Given a source file it generates a dictionary with a unique token string as a key and the identification integer as a value

    Input:
    :filePath: path to the source file
    :cutoff: optional cutoff value to eliminate words with low frequency, if not initialized ano word is cutoff

    Output:
    :tokenMapping: a mapping from unique tokens/POS-tag pairing to unique integer identifications
    '''

    print("Getting Word Dictionary...", end='')
    
    tokenMapping = {}

    # count each token
    for t in wordTokens:
        tokenMapping[t] = tokenMapping.get(t, 0) + 1

    # remove words below cutoff and set value as index
    idx = 0
    for k, v in tokenMapping.items():
        if v < cutoff:
            del tokenMapping[k]
        else:
            tokenMapping[k] = idx
            idx += 1
    tokenMapping['UNK'] = len(tokenMapping)

    print(" Done")
    return tokenMapping

In [165]:
def getTokens(fileName):
    '''
    Gets list of tokens from file seperated by spaces

    Input:
    :fileName: path to file

    Output:
    :tokenMapping: list of word tokens
    '''
    
    print("Getting Tokens...", end='')
    
    with open(fileName) as f:
        textBody = f.read()
    
    print(" Done")
    return textBody.split()

In [166]:
def getCtxMatrix(tokens, dictionary, window):
    ''' 
    Given an ordered list of tokens, creates a sparse context matrix, returned as a lil_matrix.
    
    Input:
    :tokens: list of ordered word tokens
    :dictionary: word to numbered id mapping
    :window: the window size to consider when creating context matrix
    
    Output:
    :mat: a sparse context matrix, returned as a lil_matrix
    '''
    
    dim = len(dictionary)
    
    print("Creating Container Matrix of size %d..." % dim, end='')
    
    mat = scipy.sparse.lil_matrix((dim, dim), dtype=np.int)
    for i in range(dim):
        mat[i] = np.zeros(dim, dtype=np.int)
    
    print(" Done")
    
    start = time.time()
    
    print("Building Context Matrix...", end='')
        
    for i in range(window, len(tokens) - window):
        
        lowerBound = i - window
        upperBound = i + window

        context = tokens[lowerBound:i] + tokens[i + 1:upperBound + 1]
        target = tokens[i]
        
        for c in context:
            if target in dictionary and c in dictionary:
                mat[dictionary[target], dictionary[c]] += 1
                
    print(' Finished in: %.2f sec' % (time.time() - start))
    
    return mat

In [167]:
def getPmiMatrix(ctxMatrix, cutoff_0=True):
    '''
    Given a sparse context matrix, returns a sparse matrix with the Pointwise Mutual Information Score
    
    Input:
    :ctxMatrix: context Matrix
    :cutoff: optional cutoff flag to eliminate coocurrences that have a score below 0

    Output:
    :pmiMatrix: PMI Matrix
    
    '''
    
    print('Preparing to build PMI Matrix...', end='')
    
    #step 1: get the probability of each term in the matrix - sum over columns, then sum over sums
    wordCounts = ctxMatrix.sum(axis=1)
    totalWords = wordCounts.sum()
    wordProbas = wordCounts/totalWords
    
    pmiMatrix = scipy.sparse.lil_matrix(ctxMatrix.shape)
    
    print(' Done')
    
    print('Building PMI Matrix...', end='')
    
    start = time.time()
    for i in range(ctxMatrix.shape[1]): #row-wise operation
        rowPmi = np.log(np.divide( ((ctxMatrix[i].toarray().T)/totalWords) , wordProbas*wordProbas[i] )).T

        if(cutoff_0):
            rowPmi[rowPmi<0] = 0 #0 cutoff

        pmiMatrix[i, :] = rowPmi
        del rowPmi

    print(' Finished in: %.2f sec' % (time.time()-start))
    
    return pmiMatrix

In [168]:
def batchCreator(filePath, window, batchSize, dictionary, pmiMatrix):

    '''
    Given an input string (with spaced full stops " . " to delimit sentences), creates a sparse context matrix, returned as a lil_matrix.

    Input:
    :string: input string with spaced full stop to delimit sentences
    :num_words: integer indicating the top words to keep. Pass 0 if want to pass predefined index
    :window: the window size to consider when creating context matrix
    :word_list: if num_words is 0, pass a list of words to generate a context matrix on + a 10 cutoff of rare words.

    Output:
    :matrix: a sparse context matrix, returned as a lil_matrix
    :index: a python dictionary mapping word strings to integers
    '''

    print('Creating Batches...', end='')
    start = time.time()

    batches = []
    with open(filePath) as f:
        currentBatch = np.zeros((2,batchSize), dtype=np.int)
        trail = ""
        batchIndex = 0
        for chunk in f:
            batches, currentBatch, trail, batchIndex = createBatches(trail + chunk, window, batchSize, batchIndex, batches, currentBatch, dictionary, pmiMatrix)
            if(batchIndex > 0):
                lastBatch = np.zeros((2,batchIndex), dtype=np.int)
                lastBatch[0] = currentBatch[0][:batchIndex]
                lastBatch[1] = currentBatch[1][:batchIndex]
                batches.append(lastBatch)

    print("\nFinished in: %.2f sec" % (time.time() - start))
    return batches


def createBatches(chunk, window, batchSize, batchIndex, batches, batch, dictionary, pmiMatrix):

    tokenWords = chunk.split()
    tokens = list(map(lambda t: dictionary[t] if t in dictionary else dictionary['UNK'], tokenWords))

    for i in range(window, len(tokens) - window):

        lowerBound = i - window
        upperBound = i + window

        context = tokens[lowerBound:i] + tokens[i + 1:upperBound + 1]
        target = tokens[i]

        for c in context:

            if pmiMatrix[target, c] > 0:
                batch[0][batchIndex] = target
                batch[1][batchIndex] = c
                batchIndex += 1

                if batchIndex >= batchSize:
                    print(".", end='')
                    batches.append(batch)
                    batchIndex = 0
                    batch = np.zeros((2,batchSize), dtype=np.int)

    trail = " ".join(tokenWords[len(tokens) - window:])
    return (batches, batch, trail, batchIndex)

In [169]:
def main(fileName, window, batchSize):
    tokens = getTokens(fileName)
    dictionary = makeDictionary(tokens)
    ctxMatrix = getCtxMatrix(tokens, dictionary, window)
    pmiMatrix = getPmiMatrix(ctxMatrix)
    batches = batchCreator(fileName, window, 100, dictionary, pmiMatrix)
    return batches

fileName = '../data/test'
window = 10
batchSize = 1000

main(fileName, window, batchSize)
print("Done")

Getting Tokens... Done
Getting Word Dictionary... Done
Creating Container Matrix of size 349... Done
Building Context Matrix... Finished in: 0.39 sec
Preparing to build PMI Matrix... Done
Building PMI Matrix... Finished in: 0.17 sec
Creating Batches.....................

/Users/samyzarour/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in true_divide
/Users/samyzarour/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in true_divide
/Users/samyzarour/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log
/Users/samyzarour/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in less


...................................................................
Finished in: 0.11 sec
Done
